In [1]:
# 커널 상태 확인
print("hello")

hello


In [2]:
# 배달 라이더 도착 시간 회귀분석
import pandas as pd

df = pd.read_csv('./data/rider_data.csv')
df.head()

,Distance_km,Riders_Available,Weather_Index,Rush_Hour,Time_min
0,2.934511,19,6,0,24.017860
1,6.679643,16,2,0,28.422919
2,5.257961,20,7,0,31.826773
3,4.391280,28,5,0,20.906637
4,1.514121,23,4,0,6.290465


In [3]:
# 데이터 스케일링
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(df[['Distance_km', 'Weather_Index', 'Riders_Available', 'Rush_Hour']])
scaled_features = scaler.transform(df[['Distance_km', 'Weather_Index', 'Riders_Available', 'Rush_Hour']])
scaled_features[:5] # 앞의 5개 데이터 확인
df_scaled = pd.DataFrame(scaled_features, columns=['Distance_km', 'Weather_Index', 'Riders_Available', 'Rush_Hour'])
df_scaled.head()
df_scaled['Time_min'] = df['Time_min']
df_scaled.head()


,Distance_km,Weather_Index,Riders_Available,Rush_Hour,Time_min
0,-0.396301,0.375418,0.302181,-0.656213,24.017860
1,1.576957,-0.930384,-0.118880,-0.656213,28.422919
2,0.827893,0.701869,0.442535,-0.656213,31.826773
3,0.371251,0.048968,1.565365,-0.656213,20.906637
4,-1.144685,-0.277483,0.863596,-0.656213,6.290465


In [4]:
# 배달 시간이 얼마나 걸리는지 예측하기
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

X = df[['Distance_km', 'Weather_Index', 'Riders_Available', 'Rush_Hour']]
y = df['Time_min']
X

,Distance_km,Weather_Index,Riders_Available,Rush_Hour
0,2.934511,6,19,0
1,6.679643,2,16,0
2,5.257961,7,20,0
3,4.391280,5,28,0
4,1.514121,4,23,0
...,...,...,...,...
995,1.095283,2,16,0
996,6.462538,7,12,0
997,1.389321,1,24,1
998,6.676543,0,7,0


In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(750, 4) (250, 4) (750,) (250,)


In [6]:
model = LinearRegression()
model.fit(X_train, y_train)

,"fit_intercept fit_intercept: bool, default=TrueWhether to calculate the intercept for this model. If setto False, no intercept will be used in calculations(i.e. data is expected to be centered).",True
,"copy_X copy_X: bool, default=TrueIf True, X will be copied; else, it may be overwritten.",True
,"tol tol: float, default=1e-6The precision of the solution (`coef_`) is determined by `tol` whichspecifies a different convergence criterion for the `lsqr` solver.`tol` is set as `atol` and `btol` of :func:`scipy.sparse.linalg.lsqr` whenfitting on sparse training data. This parameter has no effect when fittingon dense data... versionadded:: 1.7",1e-06
,"n_jobs n_jobs: int, default=NoneThe number of jobs to use for the computation. This will only providespeedup in case of sufficiently large problems, that is if firstly`n_targets > 1` and secondly `X` is sparse or if `positive` is setto `True`. ``None`` means 1 unless in a:obj:`joblib.parallel_backend` context. ``-1`` means using allprocessors. See :term:`Glossary ` for more details.",None
,"positive positive: bool, default=FalseWhen set to ``True``, forces the coefficients to be positive. Thisoption is only supported for dense arrays.For a comparison between a linear regression model with positive constraintson the regression coefficients and a linear regression without such constraints,see :ref:`sphx_glr_auto_examples_linear_model_plot_nnls.py`... versionadded:: 0.24",False


In [7]:
y_pred = model.predict(X_test)
y_pred


array([24.07335621, 31.98576274, 38.20466524, 14.18805152, 55.05784204,
       32.42545538, 20.55986595, 35.20623368,  9.61755837, 24.49769456,
       32.82427896, 18.90961597, 29.69666164, 29.13321193, 19.3625391 ,
       22.80615804, 37.6101647 , 21.07105959,  9.74503003,  9.56902235,
       29.69855403, 47.41472799, 21.44214995, 21.94488066, 35.55359754,
       26.93204243, 27.18268274, 28.30925827, 14.45626428, 46.27651016,
       37.28877478, 51.24450692, 32.91737961, 44.63155067,  7.02658566,
       43.94817238, 21.48903838, 38.7339735 ,  2.1258638 , 15.81697788,
       22.94298308, 18.59173363, 12.77341169, 28.3326994 ,  5.25257641,
       42.65736716, 20.40192768, 13.71220029, 21.95952186, 15.51269828,
       19.06349653, 15.27829465, 44.22729823, 27.7567622 , 11.68563675,
       33.53964048, 32.30119409, 15.71896302, 35.48261484, 32.07790742,
       41.77375375, 37.51449646, 25.55423374, 11.71657116, 27.6755575 ,
       22.24772244, 14.03943108, 10.36996377, 41.66648643, 28.52

In [8]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f'Mean Squared Error: {mse}')
print(f'R^2 Score: {r2}')

Mean Squared Error: 8.672920802368246
R^2 Score: 0.9221157222128803


In [9]:
print(model.coef_)
print(model.intercept_)

[ 4.06568289  1.44869857 -0.49664472 10.35623712]
9.903867628781782


In [10]:
coef_df = pd.DataFrame({
    'feature' : ['Distance_km', 'Weather_Index', 'Riders_Available', 'Rush_Hour'],
    'Coef' : model.coef_
})
coef_df

,feature,Coef
0,Distance_km,4.065683
1,Weather_Index,1.448699
2,Riders_Available,-0.496645
3,Rush_Hour,10.356237


In [11]:
print("절편", model.intercept_)

절편 9.903867628781782


In [12]:
# MAE, MSE, RMSE, R^2 점수 계산
from sklearn.metrics import mean_absolute_error
import numpy as np
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
print(f'Mean Absolute Error: {mae}')
print(f'Root Mean Squared Error: {rmse}')
print(f'R^2 Score: {r2}')

Mean Absolute Error: 2.3744577796800717
Root Mean Squared Error: 2.9449823093472474
R^2 Score: 0.9221157222128803


In [13]:
# Ridge, Lasso 회귀분석
from sklearn.linear_model import Ridge, Lasso
from sklearn.model_selection import GridSearchCV
ridge = Ridge()
parameters = {'alpha':[0.001, 0.01, 0.1, 1.0, 10.0, 100.0]}
ridge_reg = GridSearchCV(ridge, parameters, scoring='neg_mean_squared_error', cv=5)
ridge_reg.fit(X_train, y_train)
print("Best Ridge Alpha:", ridge_reg.best_params_)
ridge_best = ridge_reg.best_estimator_
y_ridge_pred = ridge_best.predict(X_test)
ridge_mse = mean_squared_error(y_test, y_ridge_pred)
print(f'Ridge Mean Squared Error: {ridge_mse}')
ridge_mae = mean_absolute_error(y_test, y_ridge_pred)
print(f'Ridge Mean Absolute Error: {ridge_mae}')
ridge_score = r2_score(y_test, y_ridge_pred)
print(f'Ridge R^2 Score: {ridge_score}')

Best Ridge Alpha: {'alpha': 0.1}
Ridge Mean Squared Error: 8.670500056812568
Ridge Mean Absolute Error: 2.3741455426200093
Ridge R^2 Score: 0.9221374609124035


In [14]:
lasso = Lasso()
lasso_reg = GridSearchCV(lasso, parameters, scoring='neg_mean_squared_error', cv=5)
lasso_reg.fit(X_train, y_train)
print("Best Lasso Alpha:", lasso_reg.best_params_)
lasso_best = lasso_reg.best_estimator_
y_lasso_pred = lasso_best.predict(X_test)
lasso_mse = mean_squared_error(y_test, y_lasso_pred)
print(f'Lasso Mean Squared Error: {lasso_mse}')
lasso_mae = mean_absolute_error(y_test, y_lasso_pred)
print(f'Lasso Mean Absolute Error: {lasso_mae}')
lasso_score = r2_score(y_test, y_lasso_pred)
print(f'Lasso R^2 Score: {lasso_score}')


Best Lasso Alpha: {'alpha': 0.001}
Lasso Mean Squared Error: 8.671092758060281
Lasso Mean Absolute Error: 2.374219369868547
Lasso R^2 Score: 0.9221321383561766


In [15]:
X_test

,Distance_km,Weather_Index,Riders_Available,Rush_Hour
521,2.975791,10,25,0
737,5.807896,1,6,0
740,3.507419,7,13,1
660,2.814629,5,29,0
411,6.675403,7,5,1
...,...,...,...,...
109,1.000369,1,8,0
430,1.429446,1,14,1
77,0.981290,3,7,0
84,2.521385,5,7,0
